In [1]:
def emu_list(model):
    emud = dict()
    atom_carbons = dict()
    for met, atommap in model.metabolites_info.items():
        
        if '.ex' in met:
            continue
        origatoms = atommap[0]
        if origatoms.atoms == None:
            continue
        total_atoms = origatoms.atoms[0]
        
        n_carbons = len(total_atoms)
        atom_carbons[met] = n_carbons
        
        emud[met] = ''.join([str(i) for i in range(1, n_carbons+1)])

    return (emud, atom_carbons)

In [2]:
from freeflux import Model
import random
import pandas as pd

MODEL_FILE = 'cbsimple.tsv'

model = Model('demo')
model.read_from_file(MODEL_FILE)

isim = model.simulator('inst')

EMUs = emu_list(model)[0]
num_carbons = emu_list(model)[1]
'''
isim.set_target_EMUs({
    'RUBP': '12345',
    'PP': '12345',
    'FBP': '123456',
    'F6P': '123456',
    'G6P': '123456',
    'G1P': '123456',
    'T3P': '123',
    'ADPG': '123456',
    'CO2': '1'
})
'''
isim.set_target_EMUs(EMUs)

#isim.set_target_EMUs({'rb15bp.h_12345': None})
isim.set_labeling_strategy(
    'CO2.ex', 
    labeling_pattern = ['1'], 
    percentage = [0.95], 
    purity = [1]
)

In [3]:
fluxes = {
    "T_CO2": 265.733504558412,
    "RUBISCO_CO2": 265.733504558412,
    "RUBISCO_O2": 91.8596342154520,
    "EX_2PG": 91.8596342154519,
    "GAPDHp": 623.326643331767,
    "FBAp": 901.690708753750,
    "PFPp": 132.866752279205,
    "TK3": 119.197712924601,
    "ALD": 119.197712924593,
    "SBPase": 119.197712924600,
    "TK1_f": 238.395425849188,
    "TK1_b": 0,
    "TK2_f": 119.197712924594,
    "TK2_b": 0,
    "PPI_f": 119.197712924596,
    "PPI_b": 0,
    "PRK": 357.593138773604,
    "PGIp": 556.353515250582,
    "PGMp": 942.387852878675,
    "AGP": 13.6690393545312,
    "SS": 13.6690393545314,
    "PGIp_rev": 542.684475896194,
    "PGMp_rev": 928.718813523898,
    "FBAp_rev": 768.823956475002
}

for fluxid, value in fluxes.items():
    isim.set_flux(fluxid, value)

concs = dict()
for met in model.metabolites:
    concs[met] = random.uniform(0, 10)

for concid, value in concs.items():
    isim.set_concentration(concid, value)

In [4]:
isim.set_timepoints([0, 5, 10, 20, 40])
isim.prepare(n_jobs = 1)
res = isim.simulate()

In [5]:
def mdv_to_list(mdv_obj):
    """
    Convert MDV(...) to a plain Python list, robust to different implementations.
    Works if the object is iterable, has .value, or a NumPy-like .tolist().
    """
    if hasattr(mdv_obj, "value"):
        x = mdv_obj.value
    else:
        x = mdv_obj
    try:
        return list(x)              # iterable?
    except TypeError:
        return list(x.tolist()) 
        
def build_mdv_table(res, light=None, alga=None):
    """
    Create a DataFrame where rows are timepoints and columns are concatenated
    MDV components for each EMU (e.g., E4P_0.., RUBP_0..).
    """
    # discover timepoints if not provided
    tps = [0, 5, 10, 20, 40]

    frames = []
    for emu in res.simulated_EMUs:
        label = emu.split('_')[0]  # "E4P_1234" -> "E4P"
        mdv_map = res.simulated_MDV(emu)

        # use the first timepoint to know the vector length and build column names
        first_vec = mdv_to_list(mdv_map[tps[0]])
        cols = [f"{label}_{i}" for i in range(len(first_vec))]

        data = [mdv_to_list(mdv_map[t]) for t in tps]
        df = pd.DataFrame(data, index=tps, columns=cols)
        frames.append(df)

    out = pd.concat(frames, axis=1)
    out.index.name = "LabelingTime_sec_"

    # optional metadata columns (to match your earlier table layout)
    if light is not None:
        out.insert(0, "Light", light)
    if alga is not None:
        out.insert(1 if light is not None else 0, "Alga", alga)
    out.reset_index(inplace=True)

    return out

# --- usage -----------------------------------------------------------------
# If your API exposes timepoints via keys() you can omit timepoints below.
# Otherwise, pass the ones you need explicitly:
# table = build_mdv_table(res, timepoints=[0, 5, 10, 20, 40], light="100 uE", alga="Chlamy")

table = build_mdv_table(res, light="100 uE", alga="Chlamy")
print(table)

   LabelingTime_sec_   Light    Alga        ADPG_0    ADPG_1    ADPG_2  \
0                  0  100 uE  Chlamy  9.374930e-01  0.060838  0.001645   
1                  5  100 uE  Chlamy  1.609159e-01  0.020846  0.024369   
2                 10  100 uE  Chlamy  6.268334e-04  0.002167  0.005144   
3                 20  100 uE  Chlamy  2.974693e-08  0.000003  0.000099   
4                 40  100 uE  Chlamy  1.465358e-08  0.000002  0.000081   

     ADPG_3        ADPG_4        ADPG_5        ADPG_6  ...     T3P_1  \
0  0.000024  1.924343e-07  8.325267e-10  1.500730e-12  ...  0.031417   
1  0.087165  1.552610e-01  1.554787e-01  3.959639e-01  ...  0.122048   
2  0.021355  6.595496e-02  2.263815e-01  6.783701e-01  ...  0.013124   
3  0.002278  3.098465e-02  2.308247e-01  7.358107e-01  ...  0.006994   
4  0.002079  2.996416e-02  2.303068e-01  7.375669e-01  ...  0.006977   

      T3P_2     T3P_3    x2PG_0    x2PG_1    x2PG_2   x3PGA_0   x3PGA_1  \
0  0.000340  0.000001  0.978714  0.021171  0.00

In [52]:
import re
import numpy as np
from functools import reduce
wanted_mets = ['RUBP', 'F6P', 'G6P', 'FBP', 'T3P', 'x3PGA']

def matrix(df, prefix):
    # pick only columns like ADPG_0, ADPG_1, ..., ADPG_10
    cols = [c for c in df.columns
            if isinstance(c, str) and re.fullmatch(rf'^{re.escape(prefix)}_?(\d+)$', c)]
    # sort by the numeric suffix (so _10 comes after _9)
    return df[cols].to_numpy() # (matrix, column_order)


def build_diff_mat(model, table, wanted_mets):
    
    model_S = model.get_total_stoichiometric_matrix() 
    MID_dict = {met: matrix(table, met) for met in EMUs.keys()}
    all_rxns = model_S.columns
        
    diff_mat = pd.DataFrame()

    out = {}
    
    for met in wanted_mets:
        print(met)

        n = num_carbons[met]
        diff_df = pd.DataFrame(0.0, index=range(n+1), columns=all_rxns)

        row = model_S.loc[met]
        consumers = row[row < 0].index.tolist()
        producers = row[row > 0].index.tolist()
        #v_self = MID_dict[met][0, :n]
        
        for rxn in consumers:

            rxn_col = model_S[rxn]
            produced = []
            for m, coeff in rxn_col.items():
                if coeff < 0:
                    produced.extend([m]*int(abs(coeff)))
            
            if len(produced) == 1:
                product = produced[0]
                v_self = MID_dict[product][0, :n+1]
                diff_df.loc[:, rxn] = -v_self
            else:
                vectors_c = []
                for p_in in produced:
                    if p_in not in MID_dict:
                    # if MID not available, skip (or raise)
                        continue
                    c_len = int(num_carbons[p_in])
                    vectors_c.append(MID_dict[p_in][0, :c_len+1])
                
                conv = reduce(np.convolve, vectors_c, np.array([1.0]))
                conv = conv[:n+1]
                
                diff_df.loc[:, rxn] = -conv
                
        for rxn in producers:

            rxn_col = model_S[rxn]
            consumed = []
            for m, coeff in rxn_col.items():
                if coeff > 0:
                    consumed.extend([m]*int(abs(coeff)))

            if len(consumed) == 1:
                reactant = consumed[0]
                v_another_self = MID_dict[reactant][0, :n+1]                
                diff_df.loc[:, rxn] = v_another_self
            else:
                vectors_p = []
                for m_in in consumed:
                    if m_in not in MID_dict:
                    # if MID not available, skip (or raise)
                        continue
                    c_len = int(num_carbons[m_in])
                    vectors_p.append(MID_dict[m_in][0, :c_len+1])

                conv = reduce(np.convolve, vectors_p, np.array([1.0]))
                conv = conv[:n+1]
                
                diff_df.loc[:, rxn] = conv
            
        out[met] = diff_df
        print('######################')

    return out, MID_dict
     
x, sus = build_diff_mat(model, table, wanted_mets)

for key, value in x.items():
    print(key)
    display(value)

RUBP
######################
F6P
######################
G6P
######################
FBP
######################
T3P
######################
x3PGA
######################
RUBP


,T_CO2,RUBISCO_CO2,RUBISCO_O2,EX_2PG,GAPDHp,FBAp,PFPp,TK3,ALD,SBPase,...,PPI_f,PPI_b,PRK,PGIp,PGMp,AGP,SS,PGIp_rev,PGMp_rev,FBAp_rev
0,0.0,-9.374930e-01,-9.476327e-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,9.476327e-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,-6.083802e-02,-5.124669e-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.124669e-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,-1.645019e-03,-1.108541e-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.108541e-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,-2.372277e-05,-1.198967e-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.198967e-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,-1.924343e-07,-6.483852e-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6.483852e-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,-8.325267e-10,-1.402552e-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.402552e-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0


F6P


,T_CO2,RUBISCO_CO2,RUBISCO_O2,EX_2PG,GAPDHp,FBAp,PFPp,TK3,ALD,SBPase,...,PPI_f,PPI_b,PRK,PGIp,PGMp,AGP,SS,PGIp_rev,PGMp_rev,FBAp_rev
0,0.0,0.0,0.0,0.0,0.0,0.0,9.374930e-01,-9.374930e-01,0.0,0.0,...,0.0,0.0,0.0,-9.374930e-01,0.0,0.0,0.0,9.374930e-01,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,6.083802e-02,-6.083802e-02,0.0,0.0,...,0.0,0.0,0.0,-6.083802e-02,0.0,0.0,0.0,6.083802e-02,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.645019e-03,-1.645019e-03,0.0,0.0,...,0.0,0.0,0.0,-1.645019e-03,0.0,0.0,0.0,1.645019e-03,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,2.372277e-05,-2.372277e-05,0.0,0.0,...,0.0,0.0,0.0,-2.372277e-05,0.0,0.0,0.0,2.372277e-05,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.924343e-07,-1.924343e-07,0.0,0.0,...,0.0,0.0,0.0,-1.924343e-07,0.0,0.0,0.0,1.924343e-07,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,8.325267e-10,-8.325267e-10,0.0,0.0,...,0.0,0.0,0.0,-8.325267e-10,0.0,0.0,0.0,8.325267e-10,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,1.500730e-12,-1.500730e-12,0.0,0.0,...,0.0,0.0,0.0,-1.500730e-12,0.0,0.0,0.0,1.500730e-12,0.0,0.0


G6P


...

,T_CO2,RUBISCO_CO2,RUBISCO_O2,EX_2PG,GAPDHp,FBAp,PFPp,TK3,ALD,SBPase,...,PPI_f,PPI_b,PRK,PGIp,PGMp,AGP,SS,PGIp_rev,PGMp_rev,FBAp_rev
0,0.0,0.0,0.0,0.0,0.0,9.374930e-01,-9.374930e-01,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-9.374930e-01
1,0.0,0.0,0.0,0.0,0.0,6.083802e-02,-6.083802e-02,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-6.083802e-02
2,0.0,0.0,0.0,0.0,0.0,1.645019e-03,-1.645019e-03,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.645019e-03
3,0.0,0.0,0.0,0.0,0.0,2.372277e-05,-2.372277e-05,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.372277e-05
4,0.0,0.0,0.0,0.0,0.0,1.924343e-07,-1.924343e-07,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.924343e-07
5,0.0,0.0,0.0,0.0,0.0,8.325267e-10,-8.325267e-10,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-8.325267e-10
6,0.0,0.0,0.0,0.0,0.0,1.500730e-12,-1.500730e-12,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.500730e-12


T3P


,T_CO2,RUBISCO_CO2,RUBISCO_O2,EX_2PG,GAPDHp,FBAp,PFPp,TK3,ALD,SBPase,...,PPI_f,PPI_b,PRK,PGIp,PGMp,AGP,SS,PGIp_rev,PGMp_rev,FBAp_rev
0,0.0,0.0,0.0,0.0,0.968242,-0.937493,0.0,0.0,-0.927462,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.937493
1,0.0,0.0,0.0,0.0,0.031417,-0.060838,0.0,0.0,-0.070218,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.060838
2,0.0,0.0,0.0,0.0,0.000340,-0.001645,0.0,0.0,-0.002278,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001645
3,0.0,0.0,0.0,0.0,0.000001,-0.000024,0.0,0.0,-0.000041,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000024


x3PGA


,T_CO2,RUBISCO_CO2,RUBISCO_O2,EX_2PG,GAPDHp,FBAp,PFPp,TK3,ALD,SBPase,...,PPI_f,PPI_b,PRK,PGIp,PGMp,AGP,SS,PGIp_rev,PGMp_rev,FBAp_rev
0,0.0,0.937493,0.947633,0.0,-0.968242,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.060838,0.051247,0.0,-0.031417,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.001645,0.001109,0.0,-0.000340,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000024,0.000012,0.0,-0.000001,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
sus

{'CO2': array([[0.9893  , 0.0107  ],
        [0.049465, 0.950535],
        [0.049465, 0.950535],
        [0.049465, 0.950535],
        [0.049465, 0.950535]]),
 'RUBP': array([[9.47632715e-01, 5.12466898e-02, 1.10854055e-03, 1.19896733e-05,
         6.48385246e-08, 1.40255173e-10],
        [2.13733927e-02, 1.97871744e-02, 6.79215154e-02, 1.83044283e-01,
         1.79626476e-01, 5.28247158e-01],
        [1.55969501e-04, 5.29594378e-04, 3.24198667e-03, 3.08933478e-02,
         2.06665993e-01, 7.58513109e-01],
        [3.05268239e-07, 2.89898778e-05, 1.10473586e-03, 2.11066130e-02,
         2.02098656e-01, 7.75660700e-01],
        [2.96146332e-07, 2.84539701e-05, 1.09355206e-03, 2.10138748e-02,
         2.01903065e-01, 7.75960758e-01]]),
 'x3PGA': array([[9.68242245e-01, 3.14167351e-02, 3.39794871e-04, 1.22504300e-06],
        [4.85164361e-02, 1.22394537e-01, 1.36060575e-01, 6.93028452e-01],
        [4.05869347e-04, 1.26901947e-02, 1.38154898e-01, 8.48749038e-01],
        [1.21477386e-04, 